#### Section 1.4.1 Voltage Dividers
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

#### Imports

In [1]:
from pathlib import Path
import py4spice as spi

#### Path to Ngspice and the project

In [2]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers")

#### Prepare simulation results directory

In [3]:
# Create sim_results directory if doesn't exist
RESULTS_PATH = PROJ_PATH / "sim_results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

#### Define netlists

In [4]:
# Create netlists directory and files if they don't exist
NETLISTS_PATH = PROJ_PATH / "netlists"
NETLISTS_PATH.mkdir(parents=True, exist_ok=True)  # Create the directory

# Create title.cir with content only if it doesn't exist
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
if not TITLE_FILENAME.exists():
    with open(TITLE_FILENAME, "w") as f:
        f.write("* First line in netlist must be a comment")
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)

# These netlist objects will be combined and a top netlist will be created later
dut = spi.Netlist(NETLISTS_PATH / "dut.cir")
load = spi.Netlist(NETLISTS_PATH / "load.cir")
stimulus = spi.Netlist(NETLISTS_PATH / "stimulus.cir")
supplies = spi.Netlist(NETLISTS_PATH / "supplies.cir")

# these are two files the program will create later
TOP_FILENAME = NETLISTS_PATH / "top.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

Convert easyeda_dut.cir to dut.cir

In [5]:
easyeda = spi.Netlist(NETLISTS_PATH / "easyeda_dut.cir")
easyeda.del_line_starts_with("*") # remove first line (comment)
easyeda.del_line_starts_with("veasyeda") # remove fake voltage source
easyeda.del_line_starts_with(".tran 1m 3.1415") # remove fake transient analysis
dut = easyeda
dut.write_to_file(NETLISTS_PATH / "dut.cir") # this is now the dut (device under test)

#### Create signal vectors

In [6]:
VEC_ALL = spi.Vectors("all")
VEC_ALL_EXPANDED = spi.Vectors("in out1 out2 vee")
VEC_VIN = spi.Vectors("in")
VEC_VOUT1 = spi.Vectors("out1")
VEC_VOUT2 = spi.Vectors("out2")
VEC_IN_OUT = VEC_VIN + VEC_VOUT1 + VEC_VOUT2
print(VEC_IN_OUT)

in out1 out2


#### Define analyses

In [7]:
list_of_analyses: list[spi.Analyses] = [] # create empty list. Next sections define

# 1st analysis: operating point
# for this simulation, we'll have only one analysis
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(op1)

#### Create control section

In [8]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
spi.print_section("Control File", my_control)  # print out contents
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME) # create netlist object


--- Control File ---
.control
* Timestamp: Sun Feb 11 01:17:40 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
--------------------



#### Simulate

In [9]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = title + dut + load + supplies + stimulus + control

top.write_to_file(TOP_FILENAME)
spi.print_section("top netlist", top)

# prepare simulate object, print out command, and simulate
sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run() # run the Ngspice simulation


--- top netlist ---
* two voltage dividers
r1 in out1 140
r2 in out2 132
rload1 out1 vee 100
rload2 out2 vee 50
vee vee 0 0
vin in 0 12
.control
* timestamp: sun feb 11 01:17:40 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/netlists/top.cir
-----------------------


Note: No compatibility mode selected!


Circuit: * two voltage dividers

	* two voltage dividers

     2 : .global gnd
     3 : r1 in out1 140
     4 : r2 in out2 132
     5 : rload1 out1 vee 100
     6 : rload2 out2 vee 50
     7 : vee vee 0 0
     8 : vin in 0 12
     9 : .control
    11 : set wr_singlescale
    12 : set wr_vecnames
    13 : listing
    14 : op
    15 : print line all > /workspaces/sw_pwr_book_sim/circuits/se

#### Gather simulation results and transform into more usable format

In [14]:
# Path and name of result file
RESULTS_FILENAME = RESULTS_PATH / f"{list_of_analyses[0].name}.txt"

op1_signals = spi.Signals.from_spice_table(RESULTS_FILENAME) # create signals object
op1_values = op1_signals.signals_table(["in", "out1", "out2"])
print(VEC_IN_OUT.list_out())
# print(VEC_ALL_EXPANDED.list_out)


['in', 'out1', 'out2']
